In [8]:
import pandas as pd
from datetime import datetime
import numpy as np 
pd.set_option('display.max_columns', None)
from datetime import timedelta
import sqlite3
from src.datacleaning.create_df import load_df, impelement_column_creation
from src.datacleaning.columns_from_hand_history import fill_HH_columns
from src.datacleaning.time_columns import fill_time_columns
from src.datacleaning.dependent_columns import fill_dependent_columns
from src.datacleaning.betting_columns_CALL_LAST import fill_betting_columns

ModuleNotFoundError: No module named 'columns_from_hand_history'

In [2]:
df_full = pd.read_csv('src/datacleaning/playing_around.csv')

In [3]:
def prior_actions_for_stats(df, disclude_cash=True):
    '''
    finds actions of player prior to the hand currently being played 
    '''
    if disclude_cash:    
        mask = df['TournamentNumber'] != '' # ignores cash hands for time being 
        df = df[mask]
        
    total_dic = {} # probably replace this w/ assignment directly to the dataframe 
    unique_TN = df['TournamentNumber'].unique()
    for num in unique_TN: # iterates through all unique tournament numbers
        mask = df['TournamentNumber'] == num
        tourn_df = df[mask]
        for idx, row in tourn_df.iterrows(): # go through each row of tournament 
            row_dic = {}
            mask = tourn_df.index < idx
            df_prior = tourn_df[mask] # hands that occured previously in this tournament 
            for player in row['player_names']: # looping through all players present in hand
                player_actions = []
                for dic in df_prior['action_type']: # looking at all prior hands 
                    if player in dic:
                        player_actions.append(dic[player])
                row_dic[player] = player_actions
            total_dic[idx] = row_dic
    return total_dic

In [4]:
prior_actions_for_stats(df_full)

TypeError: string indices must be integers

In [5]:
df_full

Unnamed: 0  HandHistoryId  HandNumber  PokerSiteId  \
0               0              1  3951210633            1   
1               1              2  3951211988            1   
2               2              3  3951212830            1   
3               3              4  3951214131            1   
4               4              5  3951215154            1   
...           ...            ...         ...          ...   
54299       54299          54333  4286799273            1   
54300       54300          54334  4286799339            1   
54301       54301          54335  4286799383            1   
54302       54302          54336  4286799396            1   
54303       54303          54337  4286799413            1   

                                             HandHistory HandHistoryTimestamp  \
0      ['<session sessioncode="459742">', '<general>'...  2019-10-23 21:08:08   
1      ['<session sessioncode="459742">', '<general>'...  2019-10-23 21:09:34   
2      ['<session sessioncode="459742">', '<general>'...  2019-10-23 21:11:47   
3      ['<session sessioncode="459742">', '<general>'...  2019-10-23 21:13:37   
4      ['<session sessioncode="459742">', '<general>'...  2019-10-23 21:14:31   
...                                                  ...                  ...   
54299  ['<session sessioncode="3408296">', '<general>...  2021-03-30 08:12:25   
54300  ['<session sessioncode="4917292">', '<general>...  2021-03-30 08:12:29   
54301  ['<session sessioncode="3408296">', '<general>...  2021-03-30 08:12:34   
54302  ['<session sessioncode="4917292">', '<general>...  2021-03-30 08:12:41   
54303  ['<session sessioncode="3408296">', '<general>...  2021-03-30 08:12:50   

       GameType  TournamentNumber  buyin cash_or_tourn  my_blind_anti_total  \
0             1          459742.0   1.10     Holdem NL                 20.0   
1             1          459742.0   1.10     Holdem NL                 10.0   
2             1          459742.0   1.10     Holdem NL                  0.0   
3             1          459742.0   1.10     Holdem NL                  0.0   
4             1          459742.0   1.10     Holdem NL                  0.0   
...         ...               ...    ...           ...                  ...   
54299        82        41343764.0   5.25     Holdem NL                200.0   
54300        82        41343587.0   5.25     Holdem NL                100.0   
54301        82        41343764.0   5.25     Holdem NL                100.0   
54302       112        41343587.0   5.25     Holdem NL                 96.0   
54303        82        41343764.0   5.25     Holdem NL                200.0   

           my_cards  starting_stack          tournament_type    won     bet  \
0      ['C5', 'H2']          1500.0         3-Table Sit & Go    0.0    20.0   
1      ['C9', 'S9']          1480.0         3-Table Sit & Go  640.0   420.0   
2      ['S3', 'DK']          1700.0         3-Table Sit & Go    0.0     0.0   
3      ['C9', 'D8']          1700.0         3-Table Sit & Go    0.0     0.0   
4      ['S7', 'HQ']          1700.0         3-Table Sit & Go    0.0     0.0   
...             ...             ...                      ...    ...     ...   
54299  ['HA', 'H3']          1850.0  Hyper Turbo (500 Chips)  700.0  1850.0   
54300  ['D6', 'H4']           196.0  Hyper Turbo (500 Chips)    0.0   100.0   
54301  ['C8', 'H6']           700.0  Hyper Turbo (500 Chips)    0.0   100.0   
54302  ['DA', 'H7']            96.0  Hyper Turbo (500 Chips)    0.0    96.0   
54303  ['SQ', 'S1']           600.0  Hyper Turbo (500 Chips)    0.0   600.0   

       total_players  position  table_max_players  \
0                  9  0.222222                  9   
1                  9  0.111111                  9   
2                  9  1.000000                  9   
3                  8  0.875000                  9   
4                  8  0.750000                  9   
...              ...       ...                ...   
54299              2  0.500000  